# Create Patient Level File
# 03_create_patient_level_file

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 28/09/2025 | Adrienne | Created   | Created dataset for learning models | 
| |  |  | |

# Content

* [Introduction](#introduction)

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import json_lines
import seaborn as sns

In [6]:
# readin clean datafiles
path = "../data/clean/"
#claim_df =  pd.read_pickle(path + 'claim.pkl')
#claim_df = pd.read_pickle(path + 'claim_sample.pkl')
claim_df = pd.read_pickle(path + 'claim_mini_sample.pkl')

need to limit data to those that have HCPCS and then pick a min number of claims

In [ ]:
# drop claims with no hcpcs
claim_fil_df = claim_df[~claim_df['hcpcs_ls'].apply(lambda x: len(x) == 0)]

Pick a percentile of number of claims and use that as limit

Create the following:
- col with total number of claims
- combined DRG column
- combined HCPCS column strategy:
    - separate out preventative procedures 
    - make additional lists by one list with those and one list with the other procedures
    - cardiac
    - metabiloc
    - respotory
- consider also just an indicator that there were preventiative services received
- combined ICD10
- calculate age as of current date (?)
 

In [ ]:
#TODO: create a procedure fee dataset

In [ ]:
# base model just a set of procedures - bare services to treat condition
# model with all procedures (so if you have 3 blood draws it it in there three times)
